### ⚠ IMPORTANT ⚠

Per l'execució d'aquest notebook es necessita:

**A100 GPU**

L'avaluació i el finetuning de models de LLMs és un procés que necessita molts recursos de màquina.

In [1]:
from huggingface_hub import notebook_login

notebook_login()

# Evaluación de Benchmark de un modelo afinado

En este cuaderno, lo que haremos será cargar un modelo ya afinado y realizar algunas pruebas.

Estas pruebas son algunos de los **benchmarks** por los que se suele comparar las LLMs en cuanto a capacidades.

Veremos cuáles son los resultados y los compararemos con los de un modelo sin entrenar.


### Cargamos las dependencias y el modelo. En este caso cargaremos el modelo cuantificado a 4 bits


In [2]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

Bits And Bytes config => configuración de Bits y Bytes

[BitsAndBytesConfig](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig)

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

[Mistral-7B-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)

In [5]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

### Eleuther AI Evaluation Harness

Ahora que ya tenemos nuestro modelo base cargado, necesitamos evaluarlo.

Para esto, utilizaremos una herramienta llamada [Arnés de Evaluación de Eleuther AI](https://github.com/EleutherAI/lm-evaluation-harness). Esta es una herramienta especializada para ejecutar pruebas de rendimiento en diversas tareas de lenguaje.

¿Por qué es tan importante? ¡Es la que alimenta el [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)!

In [7]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -e .

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 49978, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 49978 (delta 27), reused 13 (delta 13), pack-reused 49909 (from 3)
Receiving objects: 100% (49978/49978), 30.25 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (34510/34510), done.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 2

In [8]:
import lm_eval
from lm_eval.tasks import TaskManager
from lm_eval.models.huggingface import HFLM
eval_model = HFLM(model, batch_size=4)

In [9]:
task_manager = TaskManager()

Ya tenemos el modelo preparado y el proceso de evaluación también

> Nota: Este paso puede tardar unos 30-40 min en una GPU A100.

Evaluaremos dos benchmarks:

- [HellaSwag](https://rowanzellers.com/hellaswag/)
- [ARC Easy](https://leaderboard.allenai.org/arc_easy/submissions/get-started)
- Un subconjunto del [benchmark MMLU](https://paperswithcode.com/dataset/mmlu), en tareas de ML

Estos son benchmarks ligeros utilizados para "puntuar" los modelos entre ellos en el [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)!

Consideraremos una media simple de sus puntuaciones como la puntuación "global" del modelo base.

Puedes añadir fácilmente tareas de evaluación a `tasks`


In [10]:
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["hellaswag", "arc_easy"],
    task_manager=task_manager,
    num_fewshot=0,
    batch_size=16
)

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

hellaswag.py:   0%|          | 0.00/4.36k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/86.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

Running loglikelihood requests: 100%|██████████| 49669/49669 [47:06<00:00, 17.57it/s]


In [11]:
import pandas as pd

pd.DataFrame(results["results"])

,arc_easy,hellaswag
alias,arc_easy,hellaswag
"acc,none",0.820286,0.655646
"acc_stderr,none",0.007878,0.004742
"acc_norm,none",0.770623,0.833599
"acc_norm_stderr,none",0.008627,0.003717


In [12]:
fs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=5,
    batch_size=16,
)

README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

mmlu_no_train.py:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

data.tar:   0%|          | 0.00/166M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Running loglikelihood requests: 100%|██████████| 448/448 [02:10<00:00,  3.43it/s]


In [13]:
import pandas as pd

pd.DataFrame(fs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
alias,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.491071
"acc_stderr,none",0.04745


### Zero-Shot MMLU

In [14]:
zs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    task_manager=task_manager,
    num_fewshot=0,
    batch_size=16
)

Running loglikelihood requests: 100%|██████████| 448/448 [00:30<00:00, 14.77it/s]


In [15]:
import pandas as pd

pd.DataFrame(zs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
alias,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.5
"acc_stderr,none",0.047458


### Chain of Thought

In [16]:
cot_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_cot_zeroshot_machine_learning"],
    task_manager=task_manager,
    num_fewshot=0,
    batch_size=16,
)

Running generate_until requests: 100%|██████████| 11/11 [01:46<00:00,  9.70s/it]


In [17]:
import pandas as pd

pd.DataFrame(cot_mmlu_results["results"])

,mmlu_flan_cot_zeroshot_machine_learning
alias,mmlu_flan_cot_zeroshot_machine_learning
"exact_match,strict-match",0.090909
"exact_match_stderr,strict-match",0.090909
"exact_match,flexible-extract",0.272727
"exact_match_stderr,flexible-extract",0.140836
